In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [ ]:
%%yaml wmh_inference_config

log:
    root: /storage/vbutoi/scratch/ESE
    log_interval: 50
    track_image_level: True
    track_pixel_level: True

model:
    exp_name: 10_23_23_Dense_WMH
    num_workers: 1  

dataset:
    split: '?'
    task: '?'
    slicing: midslice 
    input_type: image 

calibration:
    bin_weightings:
        - proportional
        - uniform
    conf_interval: [0.5, 1]
    num_bins: 10

In [ ]:
%%yaml cal_metrics_cfg 

cal_metrics:
    ECE:
        func: ese.experiment.metrics.ECE
    TL_ECE:
        func: ese.experiment.metrics.TL_ECE
    TENCE:
        func: ese.experiment.metrics.TENCE

In [ ]:
from ionpy.util import dict_product, Config

# Get the options for WMH
##################################################
wmh_dataset_options = {
    'dataset.task': ['Amsterdam'],
    'dataset.split': ['val']
}

base_wmh_cfg = Config(wmh_inference_config).update(cal_metrics_cfg)

wmh_cfgs = []
for cfg_update in dict_product(wmh_dataset_options):
    new_wmh_cfg = base_wmh_cfg.update(cfg_update)
    wmh_cfgs.append(new_wmh_cfg)

In [ ]:
len(wmh_cfgs)

In [ ]:
wmh_cfgs[0]

In [ ]:
from ese.experiment.analysis.inference import get_cal_stats
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

get_cal_stats(wmh_cfgs[0])

In [ ]:
# from ionpy.slite.submit import submit_jobs
# from ese.experiment.analysis.inference import get_cal_stats

# submit_jobs(
#     project="ESE",
#     exp_name="10_09_23_WMH_and_BP_inference",
#     config_list=cfgs, 
#     job_func=get_cal_stats
#     )